# Steps involved to build the RAG system
1. use LlamaParser to read pdfs.
2. create the vector store index.
3. use different embedding models like open ai embeddings, bge and jima ai embeddings for indexing.
4. compare the performance of different LLs like GPT-4 and Mixtral-8x7B-Instruct-v0.1 during the synthesis stage.
5. Evaluate the performance.

 # 1. use LlamaParser to read pdfs.

In [1]:
import nest_asyncio

nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(
    api_key="llx-ChdKba2V2sCF1A9NMocUMT7v87rhbjHUbI05g1IzpbE96y6d",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
)

# sync
documents = parser.load_data("LICs_New_Jeevan_Shanti.pdf")


Started parsing the file under job_id cac11eca-f175-4d74-9faf-95fcacb25453


In [2]:
print(documents)

[Document(id_='6d392318-46af-44d1-b366-ca847cdafde1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# LIFE INSURANCE CORPORATION OF INDIA\n\n(Established by the Life Insurance Corporation Act, 1956)\n\nRegistration Number: 512\n\n# LIC’S NEW JEEVAN SHANTI (UIN: 512N338V05)\n\n(A Non-Linked, Non-Participating, Individual, Single Premium, Deferred Annuity Plan)\n\n# PART – A\n\nRef: NB (Address and e-mail id of Branch Office):\n\nSpace for Name and Address of Policyholder\n\nSpace for Address and e-mail id of Branch Office\n\nDear Policyholder, Date:\n\nRe: Your Policy No. _______________\n\nWe have pleasure in forwarding herewith the above policy document comprising of Part A to Part G which please find in order.\n\nWe would also like to draw your kind attention to the information mentioned in the Schedule of the Policy and the benefits available under the Policy.\n\nSome of our Plans have certain options available u

# 2 create the vector store index.
Create a function to generate FAISS indices from embeddings.
and create the vector store index using Faiss

In [3]:
import faiss
import numpy as np

def create_faiss_index(embeddings):
    embedding_dim = len(embeddings[0]['embedding'])
    index = faiss.IndexFlatL2(embedding_dim)
    
    # Convert embeddings to numpy array and add to index
    embedding_matrix = np.array([doc['embedding'] for doc in embeddings]).astype('float32')
    index.add(embedding_matrix)
    
    return index
# Example embeddings
sample_embeddings = [{'embedding': np.random.rand(512)} for _ in range(10)]

# Create the Faiss index
index = create_faiss_index(sample_embeddings)

# Print some information about the index
print("Faiss index information:")
print("Number of stored vectors:", index.ntotal)
print("Dimensionality of stored vectors:", index.d)
print("Is the index trained?", index.is_trained)

Faiss index information:
Number of stored vectors: 10
Dimensionality of stored vectors: 512
Is the index trained? True


# 3. Use different embedding models like open ai embeddings, bge and jima ai embeddings for indexing

In [5]:
def get_openai_embedding(text):
    # Replace this with actual code to get embeddings from OpenAI API
    # Example:
    # embedding = openai.api_call_to_get_embedding(text)
    embedding = "Example embedding for text: " + text  # Placeholder example
    return embedding
# Example text
text = "This is an example text."

# Get the embedding
embedding = get_openai_embedding(text)

# Print the embedding
print(embedding)
# the embedding works, this need a paid api key

Example embedding for text: This is an example text.


In [ ]:
pip install llama-index-embeddings-jinaai

In [ ]:
pip install openai==0.28

# OpenAI Embeddings

In [10]:
import openai

# Set your OpenAI API key
openai.api_key = 'sk-proj-IlQK905iTIg3sJdnEkPqT3BlbkFJJyIEnhGSYUP2nC8waSlQ'

def get_openai_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']


# BGE Embeddings

In [11]:
class HypotheticalBGEModel:
    def embed(self, text):
        # Implement the actual embedding logic here
        return np.random.rand(768).tolist()  # Placeholder for the actual embedding

bge_embedding_model = HypotheticalBGEModel()

def get_bge_embedding(text):
    return bge_embedding_model.embed(text)

bge_embeddings = [{"id": doc.doc_id, "text": doc.text, "embedding": get_bge_embedding(doc.text)} for doc in documents]
bge_index = create_faiss_index(bge_embeddings)


In [7]:
# Iterate over the list of embeddings and print them
for embedding_info in bge_embeddings:
    print(f"Document ID: {embedding_info['id']}")
    print(f"Text: {embedding_info['text']}")
    print("Embedding:", embedding_info['embedding'])
    print()


Document ID: 9393ad19-ed8b-49b2-bf33-8e6eed6aeb0a
Text: # LIFE INSURANCE CORPORATION OF INDIA

(Established by the Life Insurance Corporation Act, 1956)

Registration Number: 512

# LIC’S NEW JEEVAN SHANTI (UIN: 512N338V05)

(A Non-Linked, Non-Participating, Individual, Single Premium, Deferred Annuity Plan)

# PART – A

Ref: NB (Address and e-mail id of Branch Office):

Space for Name and Address of Policyholder

Space for Address and e-mail id of Branch Office

Dear Policyholder, Date:

Re: Your Policy No. _______________

We have pleasure in forwarding herewith the above policy document comprising of Part A to Part G which please find in order.

We would also like to draw your kind attention to the information mentioned in the Schedule of the Policy and the benefits available under the Policy.

Some of our Plans have certain options available under them. It is important that the options, if any, available under this Plan and mentioned in the Policy Document are noted carefully as it

# Jina AI Embeddings

Using the Jina AI library:

In [12]:
# Initilise with api key
import os

jinaai_api_key = "jina_1627e9117aa84e619a94c37cdd41eed5LRW3HeoRSUnJ0vTJhohV5zRptyDy"
os.environ["jina_1627e9117aa84e619a94c37cdd41eed5LRW3HeoRSUnJ0vTJhohV5zRptyDy"] = jinaai_api_key

# Embed text and queries with Jina embedding models through JinaAI API
from llama_index.embeddings.jinaai import JinaEmbedding

embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v2-base-en",
)

embeddings = embed_model.get_text_embedding("This is the text to embed")
print("Text dim:", len(embeddings))
print("Text embed:", embeddings[:5])

embeddings = embed_model.get_query_embedding("This is the query to embed")
print("Query dim:", len(embeddings))
print("Query embed:", embeddings[:5])

Text dim: 768
Text embed: [-0.28978136, -0.8387044, 1.0005425, 0.9081567, -0.6310764]
Query dim: 768
Query embed: [-0.7071669, -0.8399794, 0.75249565, 0.5297309, -0.5782742]


In [ ]:
!pip install llama_index

# 4. Compare the Performance of Different LLMs

In [ ]:
def generate_with_gpt4(prompt):
    response = openai.Completion.create(engine="gpt-4", prompt=prompt, max_tokens=150)
    return response.choices[0].text.strip()

query = "What is the significance of using different embedding models in RAG systems?"
openai_query_embedding = get_openai_embedding(query)
openai_retrieved_docs = retrieve_documents_faiss(openai_query_embedding, openai_index, openai_embeddings)

context = "\n".join([doc['text'] for doc in openai_retrieved_docs])
prompt = f"Based on the following documents:\n\n{context}\n\nAnswer the following query:\n{query}"

gpt4_response = generate_with_gpt4(prompt)
print("\nResponse using GPT-4:\n", gpt4_response)

# Mixtral-8x7B-Instruct-v0.1

In [ ]:
def generate_with_mixtral(prompt):
    # Assuming you have a function to generate responses with Mixtral
    response = mixtral_model.generate(prompt)
    return response

mixtral_response = generate_with_mixtral(prompt)
print("\nResponse using Mixtral-8x7B-Instruct-v0.1:\n", mixtral_response)


# 5. Evaluate the Performance

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Placeholder for actual performance evaluation logic
def evaluate_performance(true_responses, generated_responses):
    precision = precision_score(true_responses, generated_responses, average='weighted')
    recall = recall_score(true_responses, generated_responses, average='weighted')
    f1 = f1_score(true_responses, generated_responses, average='weighted')
    return precision, recall, f1

# Example usage
true_responses = [...]  # Should be the actual ground truth responses
gpt4_generated_responses = [...]  # Should be generated responses using GPT-4
mixtral_generated_responses = [...]  # Should be generated responses using Mixtral

gpt4_precision, gpt4_recall, gpt4_f1 = evaluate_performance(true_responses, gpt4_generated_responses)
mixtral_precision, mixtral_recall, mixtral_f1 = evaluate_performance(true_responses, mixtral_generated_responses)

print(f"GPT-4 - Precision: {gpt4_precision}, Recall: {gpt4_recall}, F1-Score: {gpt4_f1}")
print(f"Mixtral - Precision: {mixtral_precision}, Recall: {mixtral_recall}, F1-Score: {mixtral_f1}")
